# Debug notebook
This notebook is only used to troubleshoot the S1 ARD workflow processor.

## S1 ARD workflow

In [1]:
import json
from utils.prepS1 import prepareS1

def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    print("%s: %s" % (x, loaded_json[x]))
    prepareS1(**loaded_json)

## S1 ARD job queue processor

In [2]:
import rediswq

host="redis-master"
# Uncomment next two lines if you do not have Kube-DNS working.
# import os
# host = os.getenv("REDIS_SERVICE_HOST")

q = rediswq.RedisWQ(name="jobS1", host=host)
print("Worker with sessionID: " +  q.sessionID())
print("Initial queue state: empty=" + str(q.empty()))

Worker with sessionID: d572ba8d-9a9f-44d1-972f-fa14421956fb
Initial queue state: empty=False


In [ ]:
while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        print("Working on " + itemstr)
        #time.sleep(10) # Put your actual work here instead of sleep.
        process_scene(itemstr)
        q.complete(item)
    else:
        print("Waiting for work")

Working on {"in_scene": "S2A_MSIL2A_20181213T231841_N0211_R001_T58LGP_20181214T005852", "inter_dir": "/data/intermediate/"}
Will upload to S3 bucket: ['public-eo-data', 'fiji/Sentinel_2_test/']
in_scene:  S2A_MSIL2A_20181213T231841_N0211_R001_T58LGP_20181214T005852.SAFE
scene name:  S2A_MSIL2A_20181213T231841_T58LGP
tmp:  /data/intermediate/S2A_MSIL2A_20181213T231841_T58LGP_tmp/
final out:  ['public-eo-data', 'fiji/Sentinel_2_test/']
/data/intermediate/S2A_MSIL2A_20181213T231841_T58LGP_tmp/S2A_MSIL2A_20181213T231841_N0211_R001_T58LGP_20181214T005852.SAFE/GRANULE/L2A_T58LGP_A018159_20181213T231844/IMG_DATA/R60m/T58LGP_20181213T231841_B01_60m.jp2 /data/intermediate/S2A_MSIL2A_20181213T231841_T58LGP_tmp/S2A_MSIL2A_20181213T231841_T58LGP/S2A_MSIL2A_20181213T231841_T58LGP_B01_60m.tif
/data/intermediate/S2A_MSIL2A_20181213T231841_T58LGP_tmp/S2A_MSIL2A_20181213T231841_N0211_R001_T58LGP_20181214T005852.SAFE/GRANULE/L2A_T58LGP_A018159_20181213T231844/IMG_DATA/R60m/T58LGP_20181213T231841_B09_60m

In [ ]:
print("Queue empty!")